# Practice Session 08: Data streams



Author: <font color="blue">Nil Tomas Plans</font>

E-mail: <font color="blue">nil.tomas01@estudiant.upf.edu</font>

Date: <font color="blue">26/11/2024</font>

In [1]:
import io
import nltk
import gzip
import random
import statistics
import secrets
import re
import gzip

In [2]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Nil\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Nil\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

# 0. Dataset and how to iterate

In [3]:
# Leave this code as-is

INPUT_FILE = "movie_lines.tsv.gz"

In [4]:
# Leave this code as-is

POS_NOUN = 'NN'
POS_VERB = 'VB'
POS_ADJECTIVE = 'JJ'

# Producer in Python that reads a file by words that are nouns
def read_by_parts_of_speech(filename, parts_of_speech, max_words=-1, report_every=-1):
    
    # Open the input file
    with gzip.open(INPUT_FILE, "rt", encoding='utf8') as file:
        
        # Initialize counter of words to stop at max_words
        counter = 0
    
        # Iterate through lines in the file
        for line in file:
            
            elements = line.split("\t")
            
            text = ""
            if len(elements) >= 5:
                text = elements[4].strip()
                                        
            if counter > max_words and max_words != -1:
                break
                
            for sentence in nltk.sent_tokenize(text):
                
                tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
                for word in [part[0] for part in tagged if part[1] in parts_of_speech]:
                
                    counter += 1

                    # Report
                    if (report_every != -1) and (counter % report_every == 0):
                        if max_words == -1:
                            print("- Read %d words so far" % (counter))
                        else:
                            print("- Read %d/%d words so far" % (counter, max_words))

                    # Produce the word in lowercase
                    yield word.lower()

In [5]:
for word in read_by_parts_of_speech(INPUT_FILE, [POS_ADJECTIVE], max_words=30000, report_every=10000):
    # Prints 1/1000 of words
    if random.random() < 0.001:
        print("Current noun '%s'" % (word)) 

Current noun 'ten'
Current noun 'wonderful'
Current noun 'ready'
Current noun 'clear'
Current noun 'cheap'
Current noun 'other'
Current noun 'good'
Current noun 'front'
- Read 10000/30000 words so far
Current noun 'good'
Current noun 'senseless'
Current noun 'true'
Current noun 'funny'
Current noun 'fine'
Current noun 'awful'
Current noun 'good'
Current noun 'arrive'
Current noun 'last'
Current noun 'own'
Current noun 'external'
Current noun 'nuclear'
Current noun 'dear'
Current noun 'interested'
Current noun 'other'
- Read 20000/30000 words so far
Current noun 'interested'
Current noun 'many'
Current noun 'hard'
Current noun 'poor'
Current noun 'own'
Current noun 'funny'
Current noun 'penny-ante'
Current noun 'vivid'
Current noun 'm-60'
Current noun 'little'
- Read 30000/30000 words so far


# 1. Determine approximately the top-10 words

<font size="+1" color="red">Replace this cell with your code for "add_reservoir"</font>

In [6]:
def add_to_reservoir(reservoir, item, max_reservoir_size):
    if(len(reservoir)<max_reservoir_size):#if size of reservoir is less than threshold we add an item
        reservoir.append(item)

    elif len(reservoir)==max_reservoir_size:#if the reservoir is full, randomly delete an element from  reservoir and appent a new one
        reservoir.pop(random.randint(0,len(reservoir)-1))
        reservoir.append(item)

    
    assert(len(reservoir) <= max_reservoir_size)
    

<font size="+1" color="red">Replace this cell with your code for "reservoir_sampling"</font>

In [7]:
def reservoir_sampling(filename,pos_adj, reservoir_size, max_words=-1, report_every=-1):
    reservoir = []
    
    words_read = 0
    
    for word in read_by_parts_of_speech(filename,pos_adj, max_words=max_words, report_every=report_every):
        
        words_read+=1
        #probability to ignore the word
        probability_ignore=1-reservoir_size/words_read
        #if the prob is less than probability to ignore do not append or pop any element of reservoir
        if random.random() >= probability_ignore:
            add_to_reservoir(reservoir, word, reservoir_size)
 
        

    return (words_read, reservoir)

In [8]:
# Leave this code as-is

reservoir_size = 1500
(items_seen, reservoir) = reservoir_sampling(INPUT_FILE, [POS_ADJECTIVE], reservoir_size, max_words=30000, report_every=10000)

print("Number of items seen    : %d" % items_seen)
print("Number of items sampled : %d" % len(reservoir) )

- Read 10000/30000 words so far
- Read 20000/30000 words so far
- Read 30000/30000 words so far
Number of items seen    : 30001
Number of items sampled : 1500


In [9]:
# Leave this code as-is

freq = {}
for item in reservoir:
    freq[item] = reservoir.count(item)

most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:20]
for absolute_frequency, word in most_frequent_items:
    print("%d %s" % (absolute_frequency, word))

71 good
33 last
30 little
27 much
25 right
23 sorry
23 dead
20 other
19 old
18 wrong
18 own
18 next
17 sure
17 same
17 bad
16 long
16 first
16 big
15 happy
13 whole


<font size="+1" color="red">Replace this cell with your code to print the top items and their relative frequencies</font>

In [10]:

freq = {}
for item in reservoir:
    freq[item] = reservoir.count(item)

most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:20]

i=1
print("TOP 20 ITEMS MOST FREQUENT AND %")
for absolute_frequency, word in most_frequent_items:
    percentage=absolute_frequency/len(reservoir)
    print("Item %d: %s %.2f%%" % (i, word,percentage*100))
    i+=1

TOP 20 ITEMS MOST FREQUENT AND %
Item 1: good 4.73%
Item 2: last 2.20%
Item 3: little 2.00%
Item 4: much 1.80%
Item 5: right 1.67%
Item 6: sorry 1.53%
Item 7: dead 1.53%
Item 8: other 1.33%
Item 9: old 1.27%
Item 10: wrong 1.20%
Item 11: own 1.20%
Item 12: next 1.20%
Item 13: sure 1.13%
Item 14: same 1.13%
Item 15: bad 1.13%
Item 16: long 1.07%
Item 17: first 1.07%
Item 18: big 1.07%
Item 19: happy 1.00%
Item 20: whole 0.87%


<font size="+1" color="red">Increase the max limit of words so that one pass takes about 2-3 minutes to be completed. Replace this cell with your code to try different reservoir sizes. In each case, print your estimate for the relative and absolute frequency of the words in the entire dataset.</font>

In [11]:
def top_items_freq(num_words_top,reservoir_size,dataset_size):
    #reservoir sampling algorithm
    (items_seen, reservoir) = reservoir_sampling(INPUT_FILE, [POS_ADJECTIVE], reservoir_size, max_words=30000, report_every=10000)

    print("Number of items seen    : %d" % items_seen)
    print("Number of items sampled : %d" % len(reservoir) )

    #compute frequencies of the top words
    freq = {}
    for item in reservoir:
        freq[item] = reservoir.count(item)

    most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:num_words_top]

    i=1
    print(f" \n TOP {num_words_top}  ITEMS MOST FREQUENT AND %")
    for absolute_frequency, word in most_frequent_items:
        percentage=absolute_frequency/len(reservoir)
        print("Item %d: %s %.2f%%" % (i, word,percentage*100))
        
        #compute estimate in the entire dataset
        if reservoir_size!=0:
            estimate_freq_item_dataset=absolute_frequency*dataset_size/reservoir_size
        else:
            estimate_freq_item_dataset=0
        print(f"Estimate freq for item {i} estimate_freq_item_dataset",estimate_freq_item_dataset)
        i+=1

In [12]:
top_words=5
reservoir_size=[50,100,500,1000,1500]
dataset_size=3e4
for i in range(0,len(reservoir_size)):
    print("With reservoir_size: ",reservoir_size[i],"\n")
    top_items_freq(top_words,reservoir_size[i],dataset_size)
    print()

With reservoir_size:  50 

- Read 10000/30000 words so far
- Read 20000/30000 words so far
- Read 30000/30000 words so far
Number of items seen    : 30001
Number of items sampled : 50
 
 TOP 5  ITEMS MOST FREQUENT AND %
Item 1: short 4.00%
Estimate freq for item 1 estimate_freq_item_dataset 1200.0
Item 2: quick 4.00%
Estimate freq for item 2 estimate_freq_item_dataset 1200.0
Item 3: next 4.00%
Estimate freq for item 3 estimate_freq_item_dataset 1200.0
Item 4: many 4.00%
Estimate freq for item 4 estimate_freq_item_dataset 1200.0
Item 5: first 4.00%
Estimate freq for item 5 estimate_freq_item_dataset 1200.0

With reservoir_size:  100 

- Read 10000/30000 words so far
- Read 20000/30000 words so far
- Read 30000/30000 words so far
Number of items seen    : 30001
Number of items sampled : 100
 
 TOP 5  ITEMS MOST FREQUENT AND %
Item 1: little 4.00%
Estimate freq for item 1 estimate_freq_item_dataset 1200.0
Item 2: wrong 3.00%
Estimate freq for item 2 estimate_freq_item_dataset 900.0
Item 3

<font size="+1" color="red">Replace this cell with a brief commentary indicating what reservoir size you would recommend to use, and your overall conclusions.</font>

<font size="+1" color="blue">To chose a minimum reservoir size that I would recommend to use is a size where most of the top-3 words appear into two or more consecutive runs, therefore we obtain stable results. In our case, I would choose a reservoir of size 500, because on the iteration of this size and size = 1000 we obtain the same top-2 elements because the third element varies quite often on different reservoir sizes between [100,1500]. We observe also, that the same top-2 elements are constant on bigger reservoir sizes.
Therefore, as we want to choose the minimum size, I'd recommend using size=500 </font>

# 2. Determine approximately the distinct number of words

In [13]:
# Leave this code as-is

def count_trailing_zeroes(number):
    count = 0
    while number & 1 == 0:
        count += 1
        number = number >> 1
    return count

In [14]:
# Leave this code as-is

def random_hash_function():
    # We use a cryptographically safe generator for the salt of our hash function
    salt = secrets.token_bytes(32)
    return lambda string: hash(string + str(salt))

<font size="+1" color="red">Replace this cell with your code to perform the requested number of passes.</font>

In [15]:

number_of_passes = 5
estimates = []

for i in range(number_of_passes):
    # YOUR_CODE_HERE: read the file and generate an estimate
    hash_function=random_hash_function()
    R=0
    for u in read_by_parts_of_speech(INPUT_FILE, [POS_NOUN,POS_ADJECTIVE,POS_VERB], max_words=30000, report_every=10000):
        hash_value=hash_function(u)
        r=count_trailing_zeroes(abs(hash_value))
        R=max(R,r)#maximum value of r(u) seen so far
    
    estimate=2**R#Add 2^R as an estimate for the number of distinct elements u seen
    estimates.append(estimate)
    print("Estimate on pass %d: %d distinct words" % (i+1, estimate))
    
    
    


- Read 10000/30000 words so far
- Read 20000/30000 words so far
- Read 30000/30000 words so far
Estimate on pass 1: 2048 distinct words
- Read 10000/30000 words so far
- Read 20000/30000 words so far
- Read 30000/30000 words so far
Estimate on pass 2: 4096 distinct words
- Read 10000/30000 words so far
- Read 20000/30000 words so far
- Read 30000/30000 words so far
Estimate on pass 3: 16384 distinct words
- Read 10000/30000 words so far
- Read 20000/30000 words so far
- Read 30000/30000 words so far
Estimate on pass 4: 32768 distinct words
- Read 10000/30000 words so far
- Read 20000/30000 words so far
- Read 30000/30000 words so far
Estimate on pass 5: 4096 distinct words


In [16]:
# Leave this code as-is

print("* Average of estimates: %.1f" % statistics.mean(estimates))
print("* Median  of estimates: %.1f" % statistics.median(estimates))

* Average of estimates: 11878.4
* Median  of estimates: 4096.0


<font size="+1" color="red">Compute the median of average estimates in 3 separate runs of your algorithm; each run should do 10 passes over the file. Repeat this for nouns (POS_NOUN), adjectives (POS_ADJECTIVE), and verbs (POS_VERB). Replace this cell with the results you obtained in each pass, and whether the average or the median seem more appropriate for this probabilistic counting.</font>

In [17]:
def flajolet_martin_alg_avg_med(number_of_passes, POS_):
    estimates = []

    for i in range(number_of_passes):
        hash_function=random_hash_function()
        R=0
        for u in read_by_parts_of_speech(INPUT_FILE, POS_, max_words=3000, report_every=1000):
            hash_value=hash_function(u)
            r=count_trailing_zeroes(abs(hash_value))
            R=max(R,r)#maximum value of r(u) seen so far

        estimate=2**R#Add 2^R as an estimate for the number of distinct elements u seen
        estimates.append(estimate)
        print("Estimate on pass %d: %d distinct words" % (i+1, estimate))
        
    print("* Average of estimates: %.1f" % statistics.mean(estimates))
    print("* Median  of estimates: %.1f" % statistics.median(estimates))
    
    

In [18]:
print('Flajolet-Martin probabilistic counting for POS_NOUN:\n')
flajolet_martin_alg_avg_med(10, POS_NOUN)
print()
print('Flajolet-Martin probabilistic counting for POS_ADJECTIVE:\n')
flajolet_martin_alg_avg_med(10, POS_ADJECTIVE)
print()
print('Flajolet-Martin probabilistic counting for POS_VERB:\n')
flajolet_martin_alg_avg_med(10, POS_VERB)

Flajolet-Martin probabilistic counting for POS_NOUN:

- Read 1000/3000 words so far
- Read 2000/3000 words so far
- Read 3000/3000 words so far
Estimate on pass 1: 8192 distinct words
- Read 1000/3000 words so far
- Read 2000/3000 words so far
- Read 3000/3000 words so far
Estimate on pass 2: 2048 distinct words
- Read 1000/3000 words so far
- Read 2000/3000 words so far
- Read 3000/3000 words so far
Estimate on pass 3: 512 distinct words
- Read 1000/3000 words so far
- Read 2000/3000 words so far
- Read 3000/3000 words so far
Estimate on pass 4: 4096 distinct words
- Read 1000/3000 words so far
- Read 2000/3000 words so far
- Read 3000/3000 words so far
Estimate on pass 5: 8192 distinct words
- Read 1000/3000 words so far
- Read 2000/3000 words so far
- Read 3000/3000 words so far
Estimate on pass 6: 4096 distinct words
- Read 1000/3000 words so far
- Read 2000/3000 words so far
- Read 3000/3000 words so far
Estimate on pass 7: 1024 distinct words
- Read 1000/3000 words so far
- Read 

<font size="+1" color="blue">
    To avoid that the computations of average and mean lasted more than 15 minutes, I decide to put a lowest threshold of words to read (max_words=3000). Above we can observe the different values of distinct words for each pass (total of 10 passes for each type of word belonging to (noun, adjective, verb).
    
   For POS_NOUN we obtain: 
    
    * Average of estimates: 4249.6    
    * Median  of estimates: 3072.0
    
   For POS_ADJECTIVE we obtain:

    * Average of estimates: 2227.2
    * Median  of estimates: 1024.0
    
   For POS_VERB we obtain: 
    
    * Average of estimates: 1894.4    
    * Median  of estimates: 512.0
    

   After analyzing the results, in my opinion the median seems more adequate for a probabilistic counting algorithm because its more robust than the average to extreme values and outliers. Therefore, the median is a better estimator for the algorithm
    
</font>

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>